## Test a trained model
#### It is to compare the model performance on zeiss / ndpi test set.

In [1]:
import os
from random import sample, seed
import cv2
import numpy as np
from keras.models import load_model
import csv

Using TensorFlow backend.


In [2]:
path = 'E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\test_zeiss\\testset'
path_tum = path + '\\tum'
path_nor = path + '\\nor'

In [3]:
ltum_ = []
for root, dirs, files in os.walk(path_tum):
    for f in files:
        if f.endswith('.tif'):
            ltum_.append(path_tum + '\\' + f)

In [4]:
lnor_ = []
for root, dirs, files in os.walk(path_nor):
    for f in files:
        if f.endswith('.tif'):
            lnor_.append(path_nor + '\\' + f)

In [5]:
seed(1)
ltum = sample(ltum_, 100000)
lnor = sample(lnor_, 100000)

In [6]:
img_tum = np.ndarray((100000, 64, 64, 3))
img_nor = np.ndarray((100000, 64, 64, 3))
for i in range(100000):
    img_tum[i, :, :, :] = cv2.cvtColor(cv2.imread(ltum[i], -1), cv2.COLOR_BGR2RGB)/255
    img_nor[i, :, :, :] = cv2.cvtColor(cv2.imread(lnor[i], -1), cv2.COLOR_BGR2RGB)/255

In [7]:
img_test = np.vstack((img_tum, img_nor))
y_test = np.vstack((np.ones(100000, np.float32), np.zeros(100000, np.float32)))
y_test = y_test.reshape(-1,1)

In [ ]:
path_res = 'E:\\deeplearning\\Hepatocarcinomes\\models\\5x\\test_dataAmount\\0k_010'

In [8]:
model = load_model(path_res + '\\1_adam_64_lr3_best_model')

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [10]:
preds = model.predict(img_test)

In [11]:
# with open(path_res + '\\vgg16_dense_bn_prob.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile, delimiter=',')
#     writer.writerow(['tum','nor'])
#     writer.writerows(preds)
#     for l in preds:
#         writer.writerow(l)

In [12]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr, tpr, thresholds  =  roc_curve(y_test, preds[:,0])
roc_auc = auc(fpr, tpr)
# worse example (use the line y=1-x)
# for i in range(len(fpr)):
#     if fpr[i] + tpr[i] >= 1:
#         i = i -1
#         break

# Normally, we use Youden's J statistic.
i = np.argmax(tpr - fpr)

# plt.plot(fpr,tpr) to draw. 'roc_auc' records the value of auc, which can be calculated by the auc() function.
plt.plot(fpr, tpr, lw=1, label='auc = %0.3f, tpr = %0.3f, tnr = %0.3f, thres = %0.3f' % ( roc_auc, tpr[i], 1-fpr[i], thresholds[i]))
# Draw diagonal
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
# Draw perfect preformance
# plt.plot([0, 0, 1], [0, 1, 1],
#          linestyle = ':',
#          color = 'black')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.title('VGG16: ROC curve of zeiss test set')
plt.legend(loc="lower right")
#     plt.show()
plt.savefig(path_res + '\\1_adam_64_lr3_roc.jpg')
plt. close() 